In [2]:
from pathlib import Path
from typing import Dict, Tuple
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from tqdm.notebook import tqdm
import csv
from datetime import datetime
directory_path = Path("../lightning_logs/theo_cov_more_dimension")

result_csv_path = Path(f"results_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.csv")

result_dict : Dict[Tuple[int, str], float] = {}

f = open(result_csv_path, "w")
writer = csv.writer(f)
writer.writerow(["iter","model","ofv"])
f.close()

iterations = sorted([entry for entry in directory_path.iterdir() if entry.is_dir()])
for models in tqdm(iterations, desc='model', leave=False):
    iteration = models.name
    seeds = sorted(list(models.iterdir()))
    for model in tqdm(seeds, desc="seed", leave=False):
        key = (iteration, model.name)
        try:
            event = sorted(list(model.glob('event*')))[-1]
            event_acc = EventAccumulator(str(event))
            event_acc.Reload()
            scalar_data = event_acc.Scalars("val_loss")[-1].value
            result_dict[key] = float(scalar_data)
            f = open(result_csv_path, "a")
            writer = csv.writer(f)
            writer.writerow([str(iteration), model.name, scalar_data])
            f.close()
        except :
            print(key)

model:   0%|          | 0/8 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

seed:   0%|          | 0/100 [00:00<?, ?it/s]

In [21]:
from dataclasses import dataclass
import pandas as pd

@dataclass
class OFVs :
    base_model : float
    cov_1_2_3_model : float
    cov_1_2_model :float
    cov_1_3_model : float
    cov_2_3_model : float
    cov_1_model : float
    cov_2_model : float
    cov_3_model : float

    def compare_ofv(self, less_cov_model, more_cov_model):
        assert more_cov_model <= less_cov_model or  more_cov_model - less_cov_model < 3.84, print(more_cov_model-less_cov_model)

    def check_ofv(self):
        self.compare_ofv(self.cov_1_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_2_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_3_model, self.cov_1_2_3_model)

        self.compare_ofv(self.cov_1_2_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_1_3_model, self.cov_1_2_3_model)
        self.compare_ofv(self.cov_2_3_model, self.cov_1_2_3_model)

        self.compare_ofv(self.cov_1_model, self.cov_1_2_model)
        self.compare_ofv(self.cov_1_model, self.cov_1_3_model)
                         
        self.compare_ofv(self.cov_2_model, self.cov_1_2_model)
        self.compare_ofv(self.cov_2_model, self.cov_2_3_model)

        self.compare_ofv(self.cov_3_model, self.cov_1_3_model)
        self.compare_ofv(self.cov_3_model, self.cov_2_3_model)

        self.compare_ofv(self.base_model, self.cov_1_model)
        self.compare_ofv(self.base_model, self.cov_2_model)
        self.compare_ofv(self.base_model, self.cov_3_model)
    
    # def backward_elimination(self):
    #     step1 = [1,2,3]
    #     step2 = [[2,3],[1,3],[1,2]]
    #     for i in step1:
    #         for j in step2:
                
        


df =pd.read_csv('results_2024-01-28 00:40:49.csv')
df_by_model = df.groupby('model')
error_list = []
for iter, d in df_by_model:
    try :
        ofv_dict = {}
        for _, row in d.iterrows():
            ofv_dict[row['iter']] =row['ofv']
        ofvs = OFVs(**ofv_dict)
        ofvs.check_ofv()
    except :
        print(iter)
        error_list.append(iter)

error_list.__len__()

17.7713623046875
seed_42
82.75309753417969
seed_59
33.211044311523466
seed_61
51.614013671875
seed_82
4.383071899414091
seed_90


5

In [22]:
error_list

['seed_42', 'seed_59', 'seed_61', 'seed_82', 'seed_90']

In [20]:
df[df['model']=='seed_19']

,iter,model,ofv
11,base_model,seed_19,2495.098877
111,cov_1_2_3_model,seed_19,140.816162
211,cov_1_2_model,seed_19,142.351501
311,cov_1_3_model,seed_19,141.645020
411,cov_1_model,seed_19,142.363861
511,cov_2_3_model,seed_19,142.035522
611,cov_2_model,seed_19,142.344727
711,cov_3_model,seed_19,142.150269
